# Task 1: Name Screening
## Problem Description
- Detect 50 Bad actors in our customer base using public data sources
- Find as many bad actors as possible using NLP techniques to match customer names with watchlist and other relevant information given.

In [15]:
import pandas as pd
import zipfile 
import warnings
warnings.filterwarnings("ignore")

In [16]:
# unzip data
with zipfile.ZipFile("data.zip","r") as zip_ref:
    zip_ref.extractall()

# load in relevant datasets
df_kyc = pd.read_csv("UofT_nodes.csv")
df_bad = pd.read_csv("targets.simple.csv", low_memory = False)

# retain relevant columns
df_kyc_cleaned = df_kyc[['NAME','BIRTH_DT']]
df_bad_cleaned = df_bad[['name','birth_date']]

df_kyc_cleaned.rename(columns={'NAME': 'name', 'BIRTH_DT': 'birth_date'}, inplace=True)

In [17]:
# remove entried where name and birth_date are null for both datasets
df_kyc_cleaned = df_kyc_cleaned[~df_kyc_cleaned['name'].isnull()].reset_index(drop=True)
df_bad_cleaned = df_bad_cleaned[~df_bad_cleaned['name'].isnull()].reset_index(drop=True)

In [18]:
# standardize the format of the birth date columns
df_kyc_cleaned['birth_date_cleaned'] = pd.to_datetime(df_kyc_cleaned['birth_date'])
df_bad_cleaned['birth_date_cleaned'] = pd.to_datetime(df_bad_cleaned['birth_date'].str.\
                                                      split(';', n=1, expand = True)[0],
                                                      errors = 'coerce')

In [19]:
df_bad_cleaned.shape

(262426, 3)

In [20]:
df_kyc_cleaned.shape

(999340, 3)

In [21]:
df_kyc_cleaned.columns

Index(['name', 'birth_date', 'birth_date_cleaned'], dtype='object')

In [22]:
df_bad_cleaned.columns

Index(['name', 'birth_date', 'birth_date_cleaned'], dtype='object')

## Method 1: String Grouper (Name + Birth Date Matched)
https://github.com/Bergvca/string_grouper

In [23]:
import pandas as pd
# update numpy if running into string_grouper import issue
# !pip install --upgrade numpy
from string_grouper import match_strings

In [24]:
matches = match_strings(df_kyc_cleaned['name'], df_bad_cleaned['name'], n_blocks = 'auto')
matches

,left_index,left_name,similarity,right_name,right_index
0,41139,David Robinson,0.847678,David ROBINSONOVA,4257
1,315806,David Robinson,0.847678,David ROBINSONOVA,4257
2,823753,David Robinson,0.847678,David ROBINSONOVA,4257
3,398002,James Mason,1.000000,James Mason,4482
4,759344,Daniel Martin,1.000000,Daniel Martin,4898
...,...,...,...,...,...
9529,793756,"Sullivan, George Ortiz",0.801988,George Ortiz,262195
9530,111485,Ana Gonzalez,0.808516,Deanna Gonzalez,262199
9531,274916,Jon Wyatt,0.802506,Brandon Wyatt,262206
9532,790486,Linda Callahan,0.850643,Amanda Callahan,262254


In [26]:
matches_cosine = matches.merge(df_kyc_cleaned.reset_index()[['index','birth_date_cleaned']], 
                            left_on='left_index',right_on='index',how='left')\
                      .drop(['index'],axis=1)\
                      .rename(columns={'birth_date_cleaned': 'kyc_bt'})\
                      .merge(df_bad_cleaned.reset_index()[['index','birth_date_cleaned']], 
                              left_on='right_index',right_on='index',how='left')\
                      .drop(['index'],axis=1)\
                      .rename(columns={'birth_date_cleaned': 'bad_bt'})

## Method 2: FuzzyWuzzy (Name + Birth Date Matched)

In [27]:
# Intermediate outputs from previous run of nested loop below (relatively long computation)
matches_fuzzy = pd.read_csv("matches_fuzzy.csv")

In [10]:
df_kyc_cleaned['YEAR'] = df_kyc_cleaned['birth_date_cleaned'].astype('datetime64[ns]').dt.year
df_bad_cleaned['year'] = df_bad_cleaned['birth_date_cleaned'].astype('datetime64[ns]').dt.year

count = 0

namelist = [] 
namebirthlist = []
scorelist = []
targetlist = []
targetbirthlist = []

for j in tqdm(range(len(df_bad_cleaned))) : 

    # print(name_solid.iloc[i,:])
    target = df_bad_cleaned.iloc[j,:]['name']
    dob = df_bad_cleaned.iloc[j,:]['birth_date_cleaned']
    year = df_bad_cleaned.iloc[j,:]['year']
    # table_name_solid = name_solid[name_solid['birth_date_cleaned'] == i].reset_index(drop = True)
    table_node = df_kyc_cleaned[df_kyc_cleaned['birth_date_cleaned'] == dob].reset_index(drop = True)

    alist = [] # storing each score compared with all people in same brithday
    for i in table_node.name.values : 
        alist.append(fuzz.token_sort_ratio(target, i))
    # print(max(alist), alist.index(max(alist)))
    if alist == [] :
        pass
    else :
        responseName = table_node.name.values[alist.index(max(alist))]
        responseBirth = table_node.birth_dt_cleaned.values[alist.index(max(alist))]
        score = max(alist)
        if score > 60 :
            #print(f'Comes:  {target}, -----> : {responseName}, --> scoring {max(alist)}')
            namelist.append(responseName)
            namebirthlist.append(responseBirth)
            scorelist.append(max(alist))
            targetlist.append(target)
            targetbirthlist.append(dob)
    # if count == 100 : # test uing first several rows
    #     break
    # count += 1

print('ALL DONE')

print('ALL DONE')

ALL DONE


In [11]:
matches_fuzzy = pd.DataFrame({'BadActor' : targetlist, 
                       'BadActor_Birth' : targetbirthlist, 
                       'Found_Name' : namelist, 
                       'Found_Name_Birth' : namebirthlist,
                       'Score' : scorelist})

## Compile Results (Name + Birth Date Matched)

In [59]:
# Matched name and birth date from method 1
df_cosine_manual_matches = matches_cosine[matches_cosine['kyc_bt'] == matches_cosine['bad_bt']]

cosine_name_matches = df_cosine_manual_matches['left_name'].to_list()

In [60]:
# Matched name and birth date from method 2 THEN 1
cosine_fuzzy_matches = match_strings(matches_fuzzy['BadActor'], 
                                     matches_fuzzy['Found_Name'],
                                     n_blocks = 'auto')['right_Found_Name'].tolist()

In [77]:
# Matched name and birth date from method 2
# manual search through entries where Score is above 75
# matches_fuzzy[matches_fuzzy['Score'] >= 65]

fuzzy_manual_matches = [715 ,203, 916, 820, 48, 661, 1108, 739, 338, 1189, 
                        725, 606, 988, 401, 580, 1025, 102, 825, 721, 242,
                        740, 594, 741, 68, 909, 66, 1195, 317, 699, 291, 722,
                        757, 53, 489, 833, 643, 763, 616, 124, 950]

df_fuzzy_manual_matches = matches_fuzzy.filter(items=fuzzy_manual_matches, axis=0)
fuzzy_name_matches = df_fuzzy_manual_matches['Found_Name'].to_list()

In [81]:
# Merge the two lists
matches_final = df_kyc[df_kyc['NAME'].isin(fuzzy_name_matches + cosine_fuzzy_matches + cosine_name_matches)]
matches_final.to_csv('badactor_foundin_kyc_bt_match.csv')

In [86]:
# Submitted version
matches_final[["NAME","CUSTOMER_ID"]].reset_index(drop=True).to_csv("0_final_badactors_list.csv")